# DPG for BipedalWalker

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.11.0
Default GPU Device: /device:GPU:0


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym
env = gym.make('BipedalWalker-v2')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.observation_space, env.action_space

(Box(24,), Box(4,))

In [4]:
state = env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action) # take a random action
    batch.append([state, action, next_state, reward, float(done)])
    state = next_state
    if done:
        state = env.reset()

To shut the window showing the simulation, use `env.close()`.

In [5]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [6]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [7]:
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

states: 0.99992806 -0.9998551
actions: 2.40696652730306 -2.2228919665018716
rewards: 2.40696652730306 -2.2228919665018716


In [8]:
env.action_space.high, env.action_space.low

(array([1., 1., 1., 1.], dtype=float32),
 array([-1., -1., -1., -1.], dtype=float32))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [13]:
def model_input(state_size, action_size):
    #states = tf.placeholder(tf.float32, [None, *state_shape], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.float32, [None, action_size], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    training = tf.placeholder(tf.bool, [], name='training')
    return states, actions, targetQs, rewards, training

In [14]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        predictions = tf.nn.tanh(logits) # [-1, +1]

        # return actions logits
        return predictions

In [15]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [16]:
# # DDPG
# def model_loss(action_size, hidden_size, states, actions, targetQs, rates):
#     actions_preds = generator(states=states, hidden_size=hidden_size, action_size=action_size)
#     gQs = discriminator(actions=actions_preds, hidden_size=hidden_size, states=states) # nextQs/targetQs
#     gloss = -tf.reduce_mean(gQs)
#     dQs = discriminator(actions=actions, hidden_size=hidden_size, states=states, reuse=True) # Qs
#     targetQs = tf.reshape(targetQs, shape=[-1, 1]) # gQs
#     dloss = tf.reduce_mean(tf.square(dQs - targetQs)) # DQN
#     rates = tf.reshape(rates, shape=[-1, 1]) # [-1,+1]
#     dloss += tf.reduce_mean(tf.square(tf.nn.tanh(dQs) - rates)) # DQN
#     return actions_preds, gQs, gloss, dloss

In [17]:
# Adverseial Q-learning
def model_loss(action_size, hidden_size, states, actions, targetQs, rewards, training):
    actions_preds = generator(states=states, hidden_size=hidden_size, action_size=action_size, training=training)
    gQs = discriminator(actions=actions_preds, hidden_size=hidden_size, states=states, training=training) #nextQs
    dQs = discriminator(actions=actions, hidden_size=hidden_size, states=states, training=training, reuse=True)#Qs
    targetQs = tf.reshape(targetQs, shape=[-1, 1]) # gQs
    gloss = tf.reduce_mean(tf.square(gQs - targetQs)) # DQN
    dloss = tf.reduce_mean(tf.square(dQs - targetQs)) # DQN
    rewards = tf.reshape(rewards, shape=[-1, 1]) # gQs
    dloss += tf.reduce_mean(tf.square(dQs - rewards)) # DQN
    return actions_preds, gQs, gloss, dloss

In [18]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, g_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(g_learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
    return g_opt, d_opt

In [19]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, g_learning_rate, d_learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, self.rewards, self.training = model_input(
            state_size=state_size, action_size=action_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_preds, self.Qs_logits, self.g_loss, self.d_loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, targetQs=self.targetQs, 
            rewards=self.rewards, training=self.training) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, 
                                           d_loss=self.d_loss,
                                           g_learning_rate=g_learning_rate, 
                                           d_learning_rate=d_learning_rate)

In [20]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, 
                               replace=False)
        return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [21]:
env.observation_space, env.action_space

(Box(24,), Box(4,))

In [22]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 24
action_size = 4
hidden_size = 24*2             # number of units in each Q-network hidden layer
g_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e2)             # experience mini-batch size == one episode size is 1000/int(1e3) steps
gamma = 0.99                   # future reward discount

In [23]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              g_learning_rate=g_learning_rate, d_learning_rate=d_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [24]:
env.action_space.high, env.action_space.low, env.action_space.shape, \
env.reward_range, env.action_space

(array([1., 1., 1., 1.], dtype=float32),
 array([-1., -1., -1., -1.], dtype=float32),
 (4,),
 (-inf, inf),
 Box(4,))

In [25]:
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

states: 0.99992806 -0.9998551
actions: 2.40696652730306 -2.2228919665018716
rewards: 2.40696652730306 -2.2228919665018716


In [26]:
state = env.reset()
total_reward = 0
num_step = 0
for each_step in range(memory_size):
    action = env.action_space.sample() # randomness
    action = np.clip(action, -1, 1) # clipped: [-1, +1]
    next_state, reward, done, _ = env.step(action)
    rate = -1 # success rate: [-1, +1]
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory updated
    total_reward += reward # max reward 300
    state = next_state
    if done is True:
        print('Progress:', each_step/memory_size)
        state = env.reset()
        # Best 100-episode average reward was 220.62 ± 0.69. 
        # (BipedalWalker-v2 is considered "solved" 
        #  when the agent obtains an average reward of at least 300 over 100 consecutive episodes.)        
        rate = total_reward/300
        rate = np.clip(rate, -1, 1) 
        total_reward = 0 # reset
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        num_step = 0 # reset

Progress: 0.01599
Progress: 0.03199
Progress: 0.0328
Progress: 0.0334
Progress: 0.03418
Progress: 0.03473
Progress: 0.03537
Progress: 0.03601
Progress: 0.05201
Progress: 0.05283
Progress: 0.0537
Progress: 0.0697
Progress: 0.07052
Progress: 0.08652
Progress: 0.10252
Progress: 0.11852
Progress: 0.13452
Progress: 0.13558
Progress: 0.13634
Progress: 0.13714
Progress: 0.13839
Progress: 0.13886
Progress: 0.13955
Progress: 0.14011
Progress: 0.14063
Progress: 0.14135
Progress: 0.15735
Progress: 0.15834
Progress: 0.15884
Progress: 0.15921
Progress: 0.15986
Progress: 0.16046
Progress: 0.17646
Progress: 0.19246
Progress: 0.20846
Progress: 0.20959
Progress: 0.22559
Progress: 0.22627
Progress: 0.22707
Progress: 0.24307
Progress: 0.24376
Progress: 0.24518
Progress: 0.2463
Progress: 0.24701
Progress: 0.24766
Progress: 0.24821
Progress: 0.24917
Progress: 0.24964
Progress: 0.26564
Progress: 0.26638
Progress: 0.28238
Progress: 0.28316
Progress: 0.28381
Progress: 0.28477
Progress: 0.30077
Progress: 0.301

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list, gloss_list, dloss_list = [], [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model2.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        num_step = 0
        gloss_batch, dloss_batch = [], []
        state = env.reset()

        # Training steps/batches
        while True:
            action_preds = sess.run(model.actions_preds, feed_dict={model.states: state.reshape([1, -1]), 
                                                                    model.training: False})
            noise = np.random.normal(loc=0, scale=0.1, size=action_size) # randomness
            action = action_preds.reshape([-1]) + noise
            #print(action.shape, action_logits.shape, noise.shape)
            action = np.clip(action, -1, 1) # clipped
            next_state, reward, done, _ = env.step(action)
            rate = -1 # success rate: -1 to +1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # memory updated
            total_reward += reward # max reward 300
            state = next_state
            
            if done is True:
                # Best 100-episode average reward was 220.62 ± 0.69. 
                # (BipedalWalker-v2 is considered "solved" 
                #  when the agent obtains an average reward of at least 300 over 100 consecutive episodes.)        
                rate = total_reward/300
                rate = np.clip(rate, -1, 1)
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1:
                        memory.buffer[-1-idx][-1] = rate
                        
            # Training
            batch = memory.sample(batch_size)
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            #rates = np.array([each[5] for each in batch])
            nextQs_logits = sess.run(model.Qs_logits, feed_dict = {model.states: next_states, 
                                                                   model.training: False})
            #nextQs = np.max(nextQs_logits, axis=1) * (1-dones) # discrete DQN
            nextQs = nextQs_logits.reshape([-1]) * (1-dones) # continuous DPG
            targetQs = rewards + (gamma * nextQs)
            gloss, dloss, _, _ = sess.run([model.g_loss, model.d_loss, model.g_opt, model.d_opt],
                                          feed_dict = {model.states: states, 
                                                       model.actions: actions,
                                                       model.targetQs: targetQs, 
                                                       model.rewards: rewards, 
                                                       model.training: False})
            gloss_batch.append(gloss)
            dloss_batch.append(dloss)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'gloss:{:.4f}'.format(np.mean(gloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        gloss_list.append([ep, np.mean(gloss_batch)])
        dloss_list.append([ep, np.mean(dloss_batch)])
        # Break episode/epoch loop
        # Did not solve the environment. 
        # Best 100-episode average reward was 220.62 ± 0.69. 
        # (BipedalWalker-v2 is considered "solved" 
        #  when the agent obtains an average reward of at least 300 over 100 consecutive episodes.)        
        if np.mean(episode_reward) >= 300:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:-111.7980 R:-111.7980 gloss:9.8430 dloss:19.8135
Episode:1 meanR:-109.8885 R:-107.9789 gloss:13.9129 dloss:27.9426
Episode:2 meanR:-110.1274 R:-110.6054 gloss:11.2913 dloss:22.7019
Episode:3 meanR:-110.2813 R:-110.7431 gloss:7.3838 dloss:14.9343
Episode:4 meanR:-110.4448 R:-111.0987 gloss:22.0639 dloss:44.4509
Episode:5 meanR:-110.9273 R:-113.3396 gloss:19.3048 dloss:39.0162
Episode:6 meanR:-111.3878 R:-114.1509 gloss:9.0165 dloss:18.3379
Episode:7 meanR:-111.7251 R:-114.0862 gloss:12.7251 dloss:25.8868
Episode:8 meanR:-112.5053 R:-118.7471 gloss:9.8833 dloss:20.1978
Episode:9 meanR:-112.9107 R:-116.5594 gloss:16.7994 dloss:34.2500
Episode:10 meanR:-113.2801 R:-116.9732 gloss:14.6961 dloss:30.1170
Episode:11 meanR:-113.5635 R:-116.6812 gloss:15.9536 dloss:32.6494
Episode:12 meanR:-113.9803 R:-118.9827 gloss:13.1512 dloss:27.1107
Episode:13 meanR:-114.2314 R:-117.4953 gloss:16.1415 dloss:33.1753
Episode:14 meanR:-113.7577 R:-107.1259 gloss:9.9885 dloss:20.7813
Episode:15

Episode:123 meanR:-112.8843 R:-102.3972 gloss:20.7027 dloss:45.1558
Episode:124 meanR:-112.6894 R:-103.0991 gloss:14.4157 dloss:31.7215
Episode:125 meanR:-112.5409 R:-102.6059 gloss:9.9077 dloss:24.2310
Episode:126 meanR:-112.3894 R:-102.1229 gloss:15.4140 dloss:33.7348
Episode:127 meanR:-112.2359 R:-102.5483 gloss:8.9320 dloss:21.3697
Episode:128 meanR:-112.0521 R:-101.9588 gloss:10.7403 dloss:23.8944
Episode:129 meanR:-111.8483 R:-101.8554 gloss:12.9678 dloss:28.9818
Episode:130 meanR:-111.4880 R:-101.9568 gloss:10.0014 dloss:23.7242
Episode:131 meanR:-111.1039 R:-103.2578 gloss:16.6071 dloss:35.8448
Episode:132 meanR:-110.5208 R:-101.4779 gloss:14.2578 dloss:31.9372
Episode:133 meanR:-109.8111 R:-102.2405 gloss:7.6782 dloss:18.5495
Episode:134 meanR:-109.0876 R:-101.5020 gloss:11.8704 dloss:26.7631
Episode:135 meanR:-108.4740 R:-104.1691 gloss:18.4932 dloss:39.5925
Episode:136 meanR:-107.8492 R:-101.7094 gloss:21.0530 dloss:46.1926
Episode:137 meanR:-107.1734 R:-101.6227 gloss:27.12

Episode:244 meanR:-103.1080 R:-104.6888 gloss:17.1103 dloss:44.3308
Episode:245 meanR:-103.1336 R:-104.6548 gloss:18.8239 dloss:47.7800
Episode:246 meanR:-103.1832 R:-107.0947 gloss:21.8734 dloss:56.3333
Episode:247 meanR:-103.2067 R:-104.5480 gloss:19.4688 dloss:50.7278
Episode:248 meanR:-103.8401 R:-167.8759 gloss:15.6641 dloss:42.1974
Episode:249 meanR:-103.8820 R:-105.9269 gloss:17.0835 dloss:46.0811
Episode:250 meanR:-103.9054 R:-104.5138 gloss:10.8119 dloss:33.1467
Episode:251 meanR:-103.9357 R:-104.5175 gloss:15.4464 dloss:41.2037
Episode:252 meanR:-103.9486 R:-103.4316 gloss:15.7599 dloss:42.0236
Episode:253 meanR:-103.9753 R:-104.6192 gloss:14.0971 dloss:38.9043
Episode:254 meanR:-103.9944 R:-104.5655 gloss:14.3083 dloss:39.1521
Episode:255 meanR:-104.0187 R:-104.4453 gloss:10.8316 dloss:32.4312
Episode:256 meanR:-104.0456 R:-104.4681 gloss:16.2564 dloss:42.9926
Episode:257 meanR:-104.0602 R:-104.7203 gloss:13.6910 dloss:37.5733
Episode:258 meanR:-104.0939 R:-104.9933 gloss:17

Episode:365 meanR:-103.3263 R:-102.0351 gloss:10.2737 dloss:35.3552
Episode:366 meanR:-103.2894 R:-102.5570 gloss:16.2545 dloss:49.2373
Episode:367 meanR:-103.2785 R:-102.0890 gloss:15.0784 dloss:45.4551
Episode:368 meanR:-103.2652 R:-102.4729 gloss:21.4870 dloss:61.2815
Episode:369 meanR:-103.2176 R:-103.4501 gloss:7.5291 dloss:33.0521
Episode:370 meanR:-102.5571 R:-102.5559 gloss:14.0514 dloss:43.4073
Episode:371 meanR:-102.5628 R:-102.3944 gloss:17.5748 dloss:50.5206
Episode:372 meanR:-102.5657 R:-102.7551 gloss:20.0415 dloss:59.8901
Episode:373 meanR:-102.5700 R:-103.2328 gloss:13.5138 dloss:45.5287
Episode:374 meanR:-102.5657 R:-102.4697 gloss:24.3097 dloss:68.9185
Episode:375 meanR:-102.5758 R:-103.1987 gloss:18.6447 dloss:57.3796
Episode:376 meanR:-102.5793 R:-102.3283 gloss:16.4179 dloss:53.3885
Episode:377 meanR:-102.5862 R:-102.4724 gloss:14.3712 dloss:44.5990
Episode:378 meanR:-102.5805 R:-102.5108 gloss:9.2366 dloss:37.5894
Episode:379 meanR:-102.5878 R:-103.0835 gloss:13.3

Episode:486 meanR:-102.6445 R:-103.4128 gloss:11.6588 dloss:44.1914
Episode:487 meanR:-102.6501 R:-102.8482 gloss:17.6935 dloss:65.9183
Episode:488 meanR:-102.6392 R:-102.2609 gloss:12.3218 dloss:52.0328
Episode:489 meanR:-102.6463 R:-103.3405 gloss:12.3096 dloss:49.8893
Episode:490 meanR:-102.6441 R:-102.3062 gloss:17.2763 dloss:58.4499
Episode:491 meanR:-102.6449 R:-102.4207 gloss:13.2451 dloss:55.7419
Episode:492 meanR:-102.6528 R:-103.3196 gloss:16.1602 dloss:58.0833
Episode:493 meanR:-102.6507 R:-101.9060 gloss:12.7373 dloss:51.4778
Episode:494 meanR:-102.6401 R:-102.2724 gloss:13.5622 dloss:56.2041
Episode:495 meanR:-102.6329 R:-102.3819 gloss:11.5353 dloss:51.7601
Episode:496 meanR:-102.6345 R:-102.1513 gloss:9.9359 dloss:41.6999
Episode:497 meanR:-102.6360 R:-103.4862 gloss:11.7060 dloss:48.6551
Episode:498 meanR:-102.6350 R:-102.3472 gloss:16.2324 dloss:64.2932
Episode:499 meanR:-102.6325 R:-102.2825 gloss:12.7156 dloss:58.2335
Episode:500 meanR:-102.6371 R:-102.3460 gloss:9.9

...

Episode:2980 meanR:-111.5261 R:-112.3291 gloss:35.7311 dloss:131.9386
Episode:2981 meanR:-111.5131 R:-106.8554 gloss:31.1728 dloss:124.8315
Episode:2982 meanR:-111.5009 R:-110.0081 gloss:35.1291 dloss:137.3642
Episode:2983 meanR:-111.5591 R:-112.4420 gloss:39.1607 dloss:137.9511
Episode:2984 meanR:-111.6081 R:-112.9567 gloss:45.2361 dloss:157.2846
Episode:2985 meanR:-111.6522 R:-113.7831 gloss:41.6928 dloss:155.6023
Episode:2986 meanR:-111.6954 R:-112.7917 gloss:39.3066 dloss:145.2223
Episode:2987 meanR:-111.6619 R:-106.9623 gloss:39.7058 dloss:154.2882
Episode:2988 meanR:-111.7142 R:-112.7895 gloss:39.3375 dloss:147.6292
Episode:2989 meanR:-111.7173 R:-112.7309 gloss:35.1538 dloss:135.7696
Episode:2990 meanR:-111.7411 R:-114.0162 gloss:44.1279 dloss:156.0169
Episode:2991 meanR:-111.7380 R:-113.1681 gloss:42.5643 dloss:163.3975
Episode:2992 meanR:-111.7632 R:-113.7862 gloss:38.1247 dloss:139.5815
Episode:2993 meanR:-111.7830 R:-114.0845 gloss:41.8538 dloss:153.2716
Episode:2994 meanR:-

Episode:3098 meanR:-109.3461 R:-104.7654 gloss:41.4474 dloss:151.0239
Episode:3099 meanR:-109.3436 R:-105.8525 gloss:37.4882 dloss:140.2178
Episode:3100 meanR:-109.3569 R:-113.0481 gloss:39.0085 dloss:145.9366
Episode:3101 meanR:-109.3669 R:-113.3381 gloss:38.7174 dloss:145.7891
Episode:3102 meanR:-109.3756 R:-113.5452 gloss:34.6936 dloss:134.6234
Episode:3103 meanR:-109.3665 R:-104.9564 gloss:43.1108 dloss:147.6305
Episode:3104 meanR:-109.3770 R:-109.5097 gloss:37.5803 dloss:138.1886
Episode:3105 meanR:-109.4581 R:-114.1965 gloss:50.4827 dloss:173.3428
Episode:3106 meanR:-109.4763 R:-114.4505 gloss:39.0917 dloss:147.8633
Episode:3107 meanR:-109.5157 R:-111.1341 gloss:31.6452 dloss:136.1035
Episode:3108 meanR:-109.4594 R:-106.9575 gloss:37.5375 dloss:148.4724
Episode:3109 meanR:-109.5452 R:-114.0047 gloss:38.5338 dloss:147.5549
Episode:3110 meanR:-109.5493 R:-107.0229 gloss:42.4087 dloss:153.3037
Episode:3111 meanR:-109.5629 R:-114.4559 gloss:38.9036 dloss:147.4401
Episode:3112 meanR:-

Episode:3216 meanR:-109.6385 R:-113.1931 gloss:40.6125 dloss:155.2863
Episode:3217 meanR:-109.6507 R:-108.1845 gloss:43.5841 dloss:163.1862
Episode:3218 meanR:-109.7090 R:-113.3949 gloss:36.8604 dloss:144.7268
Episode:3219 meanR:-109.7512 R:-113.5758 gloss:35.9572 dloss:144.0507
Episode:3220 meanR:-109.7990 R:-113.4070 gloss:34.5718 dloss:140.5618
Episode:3221 meanR:-109.8218 R:-110.8923 gloss:43.3413 dloss:156.8935
Episode:3222 meanR:-109.8282 R:-111.2743 gloss:41.9412 dloss:159.3582
Episode:3223 meanR:-109.8313 R:-112.5905 gloss:32.9903 dloss:133.5847
Episode:3224 meanR:-109.8294 R:-106.5266 gloss:42.5937 dloss:161.0132
Episode:3225 meanR:-109.8432 R:-113.5949 gloss:44.5506 dloss:162.5470
Episode:3226 meanR:-109.8743 R:-113.1542 gloss:41.6802 dloss:159.7674
Episode:3227 meanR:-109.9426 R:-113.0562 gloss:35.1408 dloss:142.9952
Episode:3228 meanR:-110.0040 R:-113.6557 gloss:38.7770 dloss:159.8782
Episode:3229 meanR:-110.0690 R:-113.4598 gloss:39.3070 dloss:150.3785
Episode:3230 meanR:-

Episode:3334 meanR:-109.1104 R:-105.1451 gloss:36.3766 dloss:143.1476
Episode:3335 meanR:-109.0376 R:-105.8485 gloss:41.2196 dloss:153.8556
Episode:3336 meanR:-108.9840 R:-105.5834 gloss:35.5447 dloss:139.1058
Episode:3337 meanR:-108.9012 R:-105.2010 gloss:42.3553 dloss:153.3787
Episode:3338 meanR:-108.9742 R:-113.3243 gloss:45.3514 dloss:165.1019
Episode:3339 meanR:-108.9082 R:-106.3062 gloss:38.9103 dloss:152.5630
Episode:3340 meanR:-108.8161 R:-104.3262 gloss:41.1093 dloss:162.5821
Episode:3341 meanR:-108.7437 R:-106.5370 gloss:39.9204 dloss:158.6469
Episode:3342 meanR:-108.6665 R:-104.1769 gloss:35.7833 dloss:147.9346
Episode:3343 meanR:-108.5811 R:-105.2071 gloss:42.4827 dloss:152.5325
Episode:3344 meanR:-108.5105 R:-106.4171 gloss:41.3811 dloss:156.5781
Episode:3345 meanR:-108.5004 R:-105.0942 gloss:37.6010 dloss:147.5231
Episode:3346 meanR:-108.5365 R:-109.2252 gloss:37.7072 dloss:144.6406
Episode:3347 meanR:-108.5357 R:-105.6788 gloss:38.3590 dloss:150.7086
Episode:3348 meanR:-

Episode:3452 meanR:-108.0586 R:-105.8800 gloss:43.8737 dloss:167.8474
Episode:3453 meanR:-108.0648 R:-106.7122 gloss:43.0844 dloss:174.1324
Episode:3454 meanR:-108.0830 R:-107.1861 gloss:45.2908 dloss:167.8946
Episode:3455 meanR:-108.0996 R:-107.0053 gloss:39.6159 dloss:164.7795
Episode:3456 meanR:-108.0390 R:-105.9987 gloss:38.4360 dloss:162.7201
Episode:3457 meanR:-108.0257 R:-106.1410 gloss:41.3569 dloss:159.7258
Episode:3458 meanR:-108.0272 R:-106.0357 gloss:46.7504 dloss:173.0536
Episode:3459 meanR:-108.0188 R:-107.1733 gloss:42.0550 dloss:168.9498
Episode:3460 meanR:-108.0131 R:-104.7620 gloss:39.5511 dloss:162.2328
Episode:3461 meanR:-107.9382 R:-105.6617 gloss:41.1144 dloss:161.5135
Episode:3462 meanR:-107.8695 R:-106.4463 gloss:41.9832 dloss:163.5954
Episode:3463 meanR:-107.8179 R:-107.6964 gloss:42.2326 dloss:168.1625
Episode:3464 meanR:-107.8197 R:-105.7475 gloss:43.9580 dloss:170.3317
Episode:3465 meanR:-107.7560 R:-106.0328 gloss:45.8654 dloss:172.6920
Episode:3466 meanR:-

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [85]:
import gym
env = gym.make('BipedalWalker-v2')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_preds = sess.run(model.actions_preds, feed_dict={model.states: state.reshape([1, -1])})
            action = np.reshape(action_preds, [-1]) # For continuous action space
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
# End the env                
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: -103.43347330792993


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.